In [1]:
import pandas as pd
import numpy as np

In [2]:
popinc = pd.read_csv("https://media.githubusercontent.com/media/MIDS-at-Duke/pds2021-opioids-team-2-ids720/data_merging/20_intermediate_files/population_2000_2020_inc.csv?token=AVKGWHZO7ZIFQMTVRAD7HU3BQLT3S")
death = pd.read_csv("/Users/Aarushi/Duke/MIDS - Fall 2021/Fall 2021/720_IDS_PDS/pds2021-opioids-team-2-ids720/20_intermediate_files/vital_stata.csv")

In [3]:
popinc.head()

,STATE,COUNTY,STNAME,CTYNAME,Year,Population,State_Code,Median_Income_2010,Income_Error_Margin
0,1,1,Alabama,Autauga County,2000,43671.0,AL,53255.0,2420.0
1,1,3,Alabama,Baldwin County,2000,140415.0,AL,50147.0,1046.0
2,1,5,Alabama,Barbour County,2000,29038.0,AL,33219.0,1889.0
3,1,7,Alabama,Bibb County,2000,20826.0,AL,41770.0,3139.0
4,1,9,Alabama,Blount County,2000,51024.0,AL,45549.0,1804.0


In [4]:
popinc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65373 entries, 0 to 65372
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   STATE                65373 non-null  int64  
 1   COUNTY               65373 non-null  int64  
 2   STNAME               65373 non-null  object 
 3   CTYNAME              65373 non-null  object 
 4   Year                 65373 non-null  int64  
 5   Population           65339 non-null  float64
 6   State_Code           65373 non-null  object 
 7   Median_Income_2010   64512 non-null  float64
 8   Income_Error_Margin  64512 non-null  float64
dtypes: float64(3), int64(3), object(3)
memory usage: 4.5+ MB


In [5]:
death.head()

,Unnamed: 0,County,County Code,Year,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,State_Code,County_Name
0,1,"Baldwin County, AL",1003.0,2003-01-01,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,AL,Baldwin County
1,38,"Jefferson County, AL",1073.0,2003-01-01,Drug poisonings (overdose) Unintentional (X40-...,D1,37.0,AL,Jefferson County
2,54,"Mobile County, AL",1097.0,2003-01-01,Drug poisonings (overdose) Unintentional (X40-...,D1,26.0,AL,Mobile County
3,106,"Cochise County, AZ",4003.0,2003-01-01,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,AZ,Cochise County
4,109,"Coconino County, AZ",4005.0,2003-01-01,Drug poisonings (overdose) Unintentional (X40-...,D1,11.0,AZ,Coconino County


In [6]:
death["YEAR"] = pd.DatetimeIndex(death["Year"]).year

In [7]:
death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9758 entries, 0 to 9757
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       9758 non-null   int64  
 1   County                           9758 non-null   object 
 2   County Code                      9758 non-null   float64
 3   Year                             9758 non-null   object 
 4   Drug/Alcohol Induced Cause       9758 non-null   object 
 5   Drug/Alcohol Induced Cause Code  9758 non-null   object 
 6   Deaths                           9758 non-null   object 
 7   State_Code                       9758 non-null   object 
 8   County_Name                      9758 non-null   object 
 9   YEAR                             9758 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 762.5+ KB


In [8]:
death.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9758 entries, 0 to 9757
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       9758 non-null   int64  
 1   County                           9758 non-null   object 
 2   County Code                      9758 non-null   float64
 3   Year                             9758 non-null   object 
 4   Drug/Alcohol Induced Cause       9758 non-null   object 
 5   Drug/Alcohol Induced Cause Code  9758 non-null   object 
 6   Deaths                           9758 non-null   object 
 7   State_Code                       9758 non-null   object 
 8   County_Name                      9758 non-null   object 
 9   YEAR                             9758 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 762.5+ KB


In [9]:
death.drop("Unnamed: 0", axis = 1, inplace=True)

In [12]:
# Mergeing Population with Deaths
merged = death.merge(
    popinc[["CTYNAME","State_Code","Population", "Median_Income_2010", "Income_Error_Margin"]],
    how="left",
    left_on=[["State_Code", "County_Name"]],
    right_on=[["CTYNAME", "State_Code"]],
)

# merged = (
#     popn.merge(
#         raw_df[["STATE", "COUNTY", "Median_Income_2010", "Income_Error_Margin"]],
#         how="left",
#         left_on=["STNAME", "CTYNAME"],
#         right_on=["STATE", "COUNTY"],
#     )
#     .drop(columns=["STATE_y", "COUNTY_y"])
#     .rename({"STATE_x": "STATE", "COUNTY_x": "COUNTY"}, axis=1)
# )


KeyError: ['CTYNAME', 'State_Code']